In [21]:
import numpy as np
import h5py
from Code.transform import transforms

In [22]:
transforms

{'ta': [<function Code.transform.<lambda>(d)>,
  <function Code.transform.<lambda>(o, a)>],
 'ua': [<function Code.transform.<lambda>(d)>,
  <function Code.transform.<lambda>(o, a)>],
 'va': [<function Code.transform.<lambda>(d)>,
  <function Code.transform.<lambda>(o, a)>],
 'wap': [<function Code.transform.<lambda>(d)>,
  <function Code.transform.<lambda>(o, a)>],
 'hus': [<function Code.transform.<lambda>(d)>,
  <function Code.transform.<lambda>(o, a)>],
 'zeta': [<function Code.transform.<lambda>(d)>,
  <function Code.transform.<lambda>(o, a)>],
 'zg': [<function Code.transform.<lambda>(d)>,
  <function Code.transform.<lambda>(o, a)>],
 'd': [<function Code.transform.<lambda>(d)>,
  <function Code.transform.<lambda>(o, a)>],
 'ps': [<function Code.transform.<lambda>(d)>,
  <function Code.transform.<lambda>(o, a)>]}

In [ ]:
f = h5py.File('TransformedDS.hdf5','a')

In [13]:
if 'LatitudeMap' not in f:
    f.create_dataset("LatitudeMap",(32,64),dtype = 'f4')
    f['LatitudeMap'] = np.tile(
        np.linspace(-1, 1, num=32, endpoint=True, retstep=False, dtype='f4', axis=0),
        (64,1)
    ).T

(32, 64)

In [17]:
raw = h5py.File('./data/11y1burn6hres45mstep_mini/chan81_ncdf4.nc','r')
#f.create_dataset("TransformedData",(14608,81, 32, 64),dtype = 'f4')
if "TransformedData" not in f:
    f.create_dataset("TransformedData",(14608,81, 32, 64),dtype = 'f4')
    
raw.close()
    